In [ ]:
import(
    "fmt"
)

In [ ]:
var a = 101

In [ ]:
%%
fmt.Println(a, runtime.Version())

In [ ]:
func main() {
    fmt.Printf("Hello World!")
}

In [ ]:
import progressbar "github.com/schollz/progressbar/v3"

%%
bar := progressbar.NewOptions(100,
                              progressbar.OptionUseANSICodes(true),
                              progressbar.OptionShowIts(),
                              progressbar.OptionSetItsString("steps"))
for i := 0; i < 100; i++ {
    bar.Add(1)
    time.Sleep(40 * time.Millisecond)
}
fmt.Printf("\nDone\n")

In [ ]:
import "bytes"
import "github.com/janpfeifer/gonb/gonbui"
import mg "github.com/erkkah/margaid"

func mgPlot(width, height int) string {
    randomSeries := mg.NewSeries()
    rand.Seed(time.Now().Unix())
    for i := float64(0); i < 10; i++ {
        randomSeries.Add(mg.MakeValue(i+1, 200*rand.Float64()))
    }

    testSeries := mg.NewSeries()
    multiplier := 2.1
    v := 0.33
    for i := float64(0); i < 10; i++ {
        v *= multiplier
        testSeries.Add(mg.MakeValue(i+1, v))
    }

    diagram := mg.New(width, height,
        mg.WithAutorange(mg.XAxis, testSeries),
        mg.WithAutorange(mg.YAxis, testSeries),
        mg.WithAutorange(mg.Y2Axis, testSeries),
        mg.WithProjection(mg.YAxis, mg.Log),
        mg.WithInset(70),
        mg.WithPadding(2),
        mg.WithColorScheme(90),
        mg.WithBackgroundColor("#f8f8f8"),
    )

    diagram.Line(testSeries, mg.UsingAxes(mg.XAxis, mg.YAxis), mg.UsingMarker("square"), mg.UsingStrokeWidth(1))
    diagram.Smooth(testSeries, mg.UsingAxes(mg.XAxis, mg.Y2Axis), mg.UsingStrokeWidth(3.14))
    diagram.Smooth(randomSeries, mg.UsingAxes(mg.XAxis, mg.YAxis), mg.UsingMarker("filled-circle"))
    diagram.Axis(testSeries, mg.XAxis, diagram.ValueTicker('f', 0, 10), false, "X")
    diagram.Axis(testSeries, mg.YAxis, diagram.ValueTicker('f', 1, 2), true, "Y")

    diagram.Frame()
    diagram.Title("A diagram of sorts 📊 📈")
    buf := bytes.NewBuffer(nil)
    diagram.Render(buf)
    return buf.String()
}

%%
gonbui.DisplaySvg(mgPlot(640, 480))

In [ ]:
import (
    "bytes"
    "math/rand"

    "github.com/janpfeifer/gonb/gonbui"
    "gonum.org/v1/plot"
    "gonum.org/v1/plot/plotter"
    "gonum.org/v1/plot/plotutil"
    "gonum.org/v1/plot/vg"
)

// randomPoints returns some random x, y points.
func randomPoints(n int) plotter.XYs {
    pts := make(plotter.XYs, n)
    for i := range pts {
        if i == 0 {
            pts[i].X = rand.Float64()
        } else {
            pts[i].X = pts[i-1].X + rand.Float64()
        }
        pts[i].Y = pts[i].X + 10*rand.Float64()
    }
    return pts
}

func GonumPlotExample(width, height int, format string) []byte {
    rand.Seed(int64(0))

    p := plot.New()

    p.Title.Text = "Plotutil example"
    p.X.Label.Text = "X"
    p.Y.Label.Text = "Y"

    err := plotutil.AddLinePoints(p,
        "First", randomPoints(15),
        "Second", randomPoints(15),
        "Third", randomPoints(15))
    if err != nil {
        panic(err)
    }

    buf := bytes.NewBuffer(nil)
    writerTo, err := p.WriterTo(vg.Points(float64(width)), vg.Points(float64(height)), format)
    if err != nil {
        panic(err)
    }
    writerTo.WriteTo(buf)
    return buf.Bytes()
}

%%
gonbui.DisplayPng(GonumPlotExample(400, 200, "png"))

In [ ]:
%%
gonbui.DisplaySvg(string(GonumPlotExample(400, 200, "svg")))

In [ ]:
func Backward[Slice ~[]E, E string](s Slice) iter.Seq2[int, E] {
    return func(yield func(int, E) bool) {
        for i := len(s) - 1; i >= 0; i-- {
            upper := strings.ToUpper(string(s[i]))
            yield(i, E(upper))
        }
        return
    }
}
%%
sl := []string{"hello1", "world", "golang"}
for i, s := range Backward(sl) {
    fmt.Printf("%d : %s\n", i, s)
}

## go-pretty

In [ ]:
import (
    "github.com/jedib0t/go-pretty/v6/table"
    "github.com/jedib0t/go-pretty/v6/text"
)

In [ ]:
%%
t := table.NewWriter()
t.SetOutputMirror(os.Stdout)

t.SetTitle("汇总")
t.AppendHeader(table.Row{"No.", "First Name", "Last Name", "Salary"})
t.AppendRows([]table.Row{
    {1, "Arya", "Stark", 3000},
    {"", "Jon", "Snow", 2000, "You know nothing, Jon Snow!"},
})
t.AppendSeparator()
t.AppendRow([]any{300, "Tyrion", "Lannister", 5000})
t.AppendFooter(table.Row{"", "", "Total", 10000})
// t.SetStyle(table.StyleColoredBright)
t.Render()